# 台北公車 API Web 介接

In [1]:
  
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import base64
from requests import request
from pprint import pprint
import pandas as pd

import re
from math import radians, cos, sin, asin, sqrt

import csv

In [66]:
pd.options.display.max_columns = 100

In [2]:
app_id = 'd745a4be31a84263aebace7253759508'
app_key = 'fvcPGn7JE8p3ZT8RmfeoBuu2goo'

In [3]:
class Auth:
    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }

## 版本詳細資訊
[參考資料](https://ptx.transportdata.tw/MOTC?t=Bus&v=2#!/CityBus/CityBusApi_DataVersion)

In [4]:
if __name__ == '__main__':
    a = Auth(app_id, app_key)
    response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bus/DataVersion/City/Taipei?&$format=JSON', headers= a.get_auth_header())
    if response.status_code == 200:
        json = response.json()
    else:
        print(f'request failed\n{response.status_code}')

In [5]:
print(f"版本詳細資訊\nVersionID: {json.get('VersionID')}\nUpdateTime: {json.get('UpdateTime')}\nUpdateCheckTime: {json.get('UpdateCheckTime')}")

版本詳細資訊
VersionID: 1114
UpdateTime: 2021-05-13T04:22:39+08:00
UpdateCheckTime: 2021-05-28T16:00:00+08:00


## 市區公車之路線資料
[參考資料](https://ptx.transportdata.tw/MOTC?t=Bus&v=2#!/CityBus/CityBusApi_Route_1)

In [6]:
if __name__ == '__main__':
    a = Auth(app_id, app_key)
    response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bus/Route/City/Taipei?$format=JSON', headers= a.get_auth_header())
    if response.status_code == 200:
        json = response.json()
    else:
        print('request failed')

In [7]:
bus_route_image_df = pd.DataFrame(json)

In [8]:
bus_route_image_df = bus_route_image_df[['RouteUID', 'HasSubRoutes', 'RouteMapImageUrl']]

In [9]:
bus_route_image_df.loc[0]['RouteMapImageUrl']

'https://ebus.gov.taipei/MapOverview?nid=0100023400'

## 市區公車之路線站序資料
[市區公車之路線站序資料](https://ptx.transportdata.tw/MOTC?t=Bus&v=2#!/CityBus/CityBusApi_StopOfRoute_1)

In [10]:
if __name__ == '__main__':
    a = Auth(app_id, app_key)
    response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bus/StopOfRoute/City/Taipei?$format=JSON', headers= a.get_auth_header())
    if response.status_code == 200:
        json = response.json()
    else:
        print('request failed')

In [11]:
bus_route_df = pd.DataFrame(json)

In [12]:
display(bus_route_df.head(3), bus_route_df.tail(3))

,RouteUID,RouteID,RouteName,Operators,SubRouteUID,SubRouteID,SubRouteName,Direction,City,CityCode,Stops,UpdateTime,VersionID
0,TPE10132,10132,"{'Zh_tw': '234', 'En': '234'}","[{'OperatorID': '100', 'OperatorName': {'Zh_tw...",TPE101320,101320,"{'Zh_tw': '234', 'En': '234'}",0,Taipei,TPE,"[{'StopUID': 'TPE33210', 'StopID': '33210', 'S...",2021-05-13T04:22:39+08:00,1114
1,TPE10132,10132,"{'Zh_tw': '234', 'En': '234'}","[{'OperatorID': '100', 'OperatorName': {'Zh_tw...",TPE101320,101320,"{'Zh_tw': '234', 'En': '234'}",1,Taipei,TPE,"[{'StopUID': 'TPE33247', 'StopID': '33247', 'S...",2021-05-13T04:22:39+08:00,1114
2,TPE10142,10142,"{'Zh_tw': '綠1', 'En': 'G1'}","[{'OperatorID': '100', 'OperatorName': {'Zh_tw...",TPE10142,10142,"{'Zh_tw': '綠1調度站發車', 'En': 'G1'}",0,Taipei,TPE,"[{'StopUID': 'TPE191081', 'StopID': '191081', ...",2021-05-13T04:22:39+08:00,1114


,RouteUID,RouteID,RouteName,Operators,SubRouteUID,SubRouteID,SubRouteName,Direction,City,CityCode,Stops,UpdateTime,VersionID
1081,TPE18508,18508,"{'Zh_tw': '棕20預', 'En': 'BR20B'}","[{'OperatorID': '1100', 'OperatorName': {'Zh_t...",TPE160469,160469,"{'Zh_tw': '棕20預', 'En': 'BR20B'}",1,Taipei,TPE,"[{'StopUID': 'TPE207232', 'StopID': '207232', ...",2021-05-13T04:22:39+08:00,1114
1082,TPE810,810,"{'Zh_tw': '敦化幹線', 'En': 'Dunhua Metro Bus'}","[{'OperatorID': '800', 'OperatorName': {'Zh_tw...",TPE810,810,"{'Zh_tw': '敦化幹線', 'En': 'Duhua Metro Bus'}",0,Taipei,TPE,"[{'StopUID': 'TPE18101', 'StopID': '18101', 'S...",2021-05-13T04:22:39+08:00,1114
1083,TPE810,810,"{'Zh_tw': '敦化幹線', 'En': 'Dunhua Metro Bus'}","[{'OperatorID': '800', 'OperatorName': {'Zh_tw...",TPE810,810,"{'Zh_tw': '敦化幹線', 'En': 'Duhua Metro Bus'}",1,Taipei,TPE,"[{'StopUID': 'TPE18181', 'StopID': '18181', 'S...",2021-05-13T04:22:39+08:00,1114


In [13]:
bus_route_df = bus_route_df[['RouteUID', 'RouteName', 'SubRouteUID', 'SubRouteName', 'Direction', 'City', 'Stops']]

In [14]:
route_name_df = pd.DataFrame(bus_route_df['RouteName'].tolist())
route_name_df.columns = ['route_name_zh_tw', 'route_name_en']
subroute_name_df = pd.DataFrame(bus_route_df['SubRouteName'].tolist())
subroute_name_df.columns = ['subroute_name_zh_tw', 'subroute_name_en']

In [15]:
bus_route_df.drop(['RouteName', 'SubRouteName'], axis=1, inplace=True)

In [16]:
bus_route_df = pd.concat([bus_route_df, route_name_df], axis=1)
bus_route_df = pd.concat([bus_route_df, subroute_name_df], axis=1)

In [17]:
bus_route_df.head()

,RouteUID,SubRouteUID,Direction,City,Stops,route_name_zh_tw,route_name_en,subroute_name_zh_tw,subroute_name_en
0,TPE10132,TPE101320,0,Taipei,"[{'StopUID': 'TPE33210', 'StopID': '33210', 'S...",234,234,234,234
1,TPE10132,TPE101320,1,Taipei,"[{'StopUID': 'TPE33247', 'StopID': '33247', 'S...",234,234,234,234
2,TPE10142,TPE10142,0,Taipei,"[{'StopUID': 'TPE191081', 'StopID': '191081', ...",綠1,G1,綠1調度站發車,G1
3,TPE10142,TPE10142,1,Taipei,"[{'StopUID': 'TPE16915', 'StopID': '16915', 'S...",綠1,G1,綠1調度站發車,G1
4,TPE10142,TPE157758,0,Taipei,"[{'StopUID': 'TPE16853', 'StopID': '16853', 'S...",綠1,G1,綠1捷運新店站發車,G1


In [18]:
# dict.get(): Return the value for key if key is in the dictionary,
#             else None(default). So this method never raises a KeyError.
def clean_stop_info(stop_info: list):
    li = []
    dic = {}
    for stop in stop_info:
        dic = {
            'StopUID': stop.get('StopUID'),
            'StopSequence': stop.get('StopSequence'),
        }

        li.append(dic)

    return li

In [19]:
bus_route_df['stop_info'] = bus_route_df['Stops'].apply(clean_stop_info)

In [20]:
bus_route_df.head()

,RouteUID,SubRouteUID,Direction,City,Stops,route_name_zh_tw,route_name_en,subroute_name_zh_tw,subroute_name_en,stop_info
0,TPE10132,TPE101320,0,Taipei,"[{'StopUID': 'TPE33210', 'StopID': '33210', 'S...",234,234,234,234,"[{'StopUID': 'TPE33210', 'StopSequence': 1}, {..."
1,TPE10132,TPE101320,1,Taipei,"[{'StopUID': 'TPE33247', 'StopID': '33247', 'S...",234,234,234,234,"[{'StopUID': 'TPE33247', 'StopSequence': 1}, {..."
2,TPE10142,TPE10142,0,Taipei,"[{'StopUID': 'TPE191081', 'StopID': '191081', ...",綠1,G1,綠1調度站發車,G1,"[{'StopUID': 'TPE191081', 'StopSequence': 1}, ..."
3,TPE10142,TPE10142,1,Taipei,"[{'StopUID': 'TPE16915', 'StopID': '16915', 'S...",綠1,G1,綠1調度站發車,G1,"[{'StopUID': 'TPE16915', 'StopSequence': 1}, {..."
4,TPE10142,TPE157758,0,Taipei,"[{'StopUID': 'TPE16853', 'StopID': '16853', 'S...",綠1,G1,綠1捷運新店站發車,G1,"[{'StopUID': 'TPE16853', 'StopSequence': 1}, {..."


In [21]:
bus_route_df.drop(['Stops'], axis=1, inplace=True)

In [22]:
def get_route_stop(list_of_dic: int):
    res = [dic.get('StopUID') for dic in list_of_dic]
    return res

In [23]:
bus_route_df['stop_in_route'] = bus_route_df['stop_info'].apply(get_route_stop)

## 市區公車之站牌資料

In [24]:
if __name__ == '__main__':
    a = Auth(app_id, app_key)
    response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bus/Stop/City/Taipei?$format=JSON', headers= a.get_auth_header())
    if response.status_code == 200:
        json = response.json()
    else:
        print('request failed')

In [25]:
bus_stop_df = pd.DataFrame(json)

In [26]:
bus_stop_df.head()

,StopUID,StopID,AuthorityID,StopName,StopPosition,StopAddress,Bearing,StationID,City,CityCode,LocationCityCode,UpdateTime,VersionID
0,TPE10000,10000,004,"{'Zh_tw': '蘆洲總站', 'En': 'Luzhou Bus Terminal'}","{'PositionLon': 121.465845849213, 'PositionLat...",中正路333號同向(總站旁候車亭)(向南),E,3698,Taipei,TPE,NWT,2021-05-13T04:22:39+08:00,1114
1,TPE10001,10001,004,"{'Zh_tw': '南港分局(重陽)', 'En': 'Nangang Police Di...","{'PositionLon': 121.596717, 'PositionLat': 25....",重陽路30號(向東),E,8627,Taipei,TPE,TPE,2021-05-13T04:22:39+08:00,1114
2,TPE10002,10002,004,"{'Zh_tw': '王爺廟口', 'En': 'Wangye Temple Entrance'}","{'PositionLon': 121.46664, 'PositionLat': 25.0...",中正路295號同向,S,3655,Taipei,TPE,NWT,2021-05-13T04:22:39+08:00,1114
3,TPE10003,10003,004,"{'Zh_tw': '中國電視公司', 'En': 'China Television Co...","{'PositionLon': 121.59918, 'PositionLat': 25.0...",重陽路120號同向,E,1612,Taipei,TPE,TPE,2021-05-13T04:22:39+08:00,1114
4,TPE10004,10004,004,"{'Zh_tw': '空中大學', 'En': 'National Open Univ.'}","{'PositionLon': 121.4677982, 'PositionLat': 25...",中正路261號同向,E,3670,Taipei,TPE,NWT,2021-05-13T04:22:39+08:00,1114


In [27]:
bus_stop_df = bus_stop_df[['StopUID', 'StopName', 'StopPosition', 'StationID', 'City']]

In [28]:
name_df = pd.DataFrame(bus_stop_df['StopName'].tolist())
name_df.columns = ['stop_name_zh_tw', 'stop_name_en']
position_df = pd.DataFrame(bus_stop_df['StopPosition'].tolist())                                      
position_df.columns = ['PositionLon', 'PositionLat', 'GeoHash']

In [29]:
bus_stop_df.drop(['StopName', 'StopPosition'], axis=1, inplace=True)

In [30]:
bus_stop_df = pd.concat([bus_stop_df, name_df], axis=1)
bus_stop_df = pd.concat([bus_stop_df, position_df], axis=1)

In [31]:
bus_stop_df

,StopUID,StationID,City,stop_name_zh_tw,stop_name_en,PositionLon,PositionLat,GeoHash
0,TPE10000,3698,Taipei,蘆洲總站,Luzhou Bus Terminal,121.465846,25.089107,wsqqsp2dx
1,TPE10001,8627,Taipei,南港分局(重陽),Nangang Police Dist.(Chongyang),121.596717,25.055561,wsqqx10p0
2,TPE10002,3655,Taipei,王爺廟口,Wangye Temple Entrance,121.466640,25.088140,wsqqsp1m1
3,TPE10003,1612,Taipei,中國電視公司,China Television Company,121.599180,25.056460,wsqqx13u6
4,TPE10004,3670,Taipei,空中大學,National Open Univ.,121.467798,25.086711,wsqqsnfhs
...,...,...,...,...,...,...,...,...
27287,TPE59942,1000541,Taipei,萬壽橋頭(木柵),Wanshou Qiaotou（Muzha）,121.571115,24.991314,wsqqnmt2u
27288,TPE59943,1000521,Taipei,萬壽橋頭(木柵),Wanshou Qiaotou（Muzha）,121.570973,24.991309,wsqqnmt2b
27289,TPE59944,1000541,Taipei,萬壽橋頭(木柵),Wanshou Qiaotou（Muzha）,121.571115,24.991314,wsqqnmt2u
27290,TPE59945,1000521,Taipei,萬壽橋頭(木柵),Wanshou Qiaotou（Muzha）,121.570973,24.991309,wsqqnmt2b


## Merge bus stop and bus route dataframe

In [32]:
bus_route_df.head()

,RouteUID,SubRouteUID,Direction,City,route_name_zh_tw,route_name_en,subroute_name_zh_tw,subroute_name_en,stop_info,stop_in_route
0,TPE10132,TPE101320,0,Taipei,234,234,234,234,"[{'StopUID': 'TPE33210', 'StopSequence': 1}, {...","[TPE33210, TPE33211, TPE33212, TPE33213, TPE33..."
1,TPE10132,TPE101320,1,Taipei,234,234,234,234,"[{'StopUID': 'TPE33247', 'StopSequence': 1}, {...","[TPE33247, TPE33248, TPE33249, TPE33250, TPE33..."
2,TPE10142,TPE10142,0,Taipei,綠1,G1,綠1調度站發車,G1,"[{'StopUID': 'TPE191081', 'StopSequence': 1}, ...","[TPE191081, TPE56987, TPE16823, TPE16825, TPE1..."
3,TPE10142,TPE10142,1,Taipei,綠1,G1,綠1調度站發車,G1,"[{'StopUID': 'TPE16915', 'StopSequence': 1}, {...","[TPE16915, TPE16917, TPE16919, TPE57417, TPE16..."
4,TPE10142,TPE157758,0,Taipei,綠1,G1,綠1捷運新店站發車,G1,"[{'StopUID': 'TPE16853', 'StopSequence': 1}, {...","[TPE16853, TPE16855, TPE16857, TPE16859, TPE16..."


In [33]:
def flattenColumn(df, column):
    """
    column is a string of the column's name.
    for each value of the column's element (which might be a list),
    duplicate the rest of columns at the corresponding row with the (each) value.
    """
    column_flat = pd.DataFrame(
        [
            [i, c_flattened]
            for i, y in df[column].apply(list).iteritems()
            for c_flattened in y
        ],
        columns=['I', column]
    )
    column_flat = column_flat.set_index('I')
    return (
        df.drop(column, 1)
             .merge(column_flat, left_index=True, right_index=True)
    )

In [34]:
bus_route_df = flattenColumn(bus_route_df, 'stop_in_route')

In [35]:
final_df = bus_stop_df.merge(bus_route_df, left_on='StopUID', right_on='stop_in_route', how='left')

In [36]:
final_df.head()

,StopUID,StationID,City_x,stop_name_zh_tw,stop_name_en,PositionLon,PositionLat,GeoHash,RouteUID,SubRouteUID,Direction,City_y,route_name_zh_tw,route_name_en,subroute_name_zh_tw,subroute_name_en,stop_info,stop_in_route
0,TPE10000,3698,Taipei,蘆洲總站,Luzhou Bus Terminal,121.465846,25.089107,wsqqsp2dx,TPE10443,TPE10443,0,Taipei,225,225,225,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10000
1,TPE10000,3698,Taipei,蘆洲總站,Luzhou Bus Terminal,121.465846,25.089107,wsqqsp2dx,TPE10443,TPE159519,0,Taipei,225,225,225狗狗公車,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10000
2,TPE10001,8627,Taipei,南港分局(重陽),Nangang Police Dist.(Chongyang),121.596717,25.055561,wsqqx10p0,TPE10231,TPE10231,0,Taipei,民權幹線,Minquan Metro Bus,民權幹線,Minquan Metro Bus,"[{'StopUID': 'TPE151077', 'StopSequence': 1}, ...",TPE10001
3,TPE10002,3655,Taipei,王爺廟口,Wangye Temple Entrance,121.466640,25.088140,wsqqsp1m1,TPE10443,TPE10443,0,Taipei,225,225,225,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10002
4,TPE10002,3655,Taipei,王爺廟口,Wangye Temple Entrance,121.466640,25.088140,wsqqsp1m1,TPE10443,TPE159519,0,Taipei,225,225,225狗狗公車,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10002


In [37]:
def get_order_in_route(StopUID, stop_info: int):
    for dic in stop_info:
        if dic.get('StopUID')==StopUID:
            res = dic.get('StopSequence')
        else:
            pass
    return res

In [38]:
final_df['order_in_route'] = final_df[['StopUID' , 'stop_info']].apply(lambda x: get_order_in_route(x['StopUID'], x['stop_info']), axis=1)

In [39]:
final_df.head()

,StopUID,StationID,City_x,stop_name_zh_tw,stop_name_en,PositionLon,PositionLat,GeoHash,RouteUID,SubRouteUID,Direction,City_y,route_name_zh_tw,route_name_en,subroute_name_zh_tw,subroute_name_en,stop_info,stop_in_route,order_in_route
0,TPE10000,3698,Taipei,蘆洲總站,Luzhou Bus Terminal,121.465846,25.089107,wsqqsp2dx,TPE10443,TPE10443,0,Taipei,225,225,225,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10000,10
1,TPE10000,3698,Taipei,蘆洲總站,Luzhou Bus Terminal,121.465846,25.089107,wsqqsp2dx,TPE10443,TPE159519,0,Taipei,225,225,225狗狗公車,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10000,10
2,TPE10001,8627,Taipei,南港分局(重陽),Nangang Police Dist.(Chongyang),121.596717,25.055561,wsqqx10p0,TPE10231,TPE10231,0,Taipei,民權幹線,Minquan Metro Bus,民權幹線,Minquan Metro Bus,"[{'StopUID': 'TPE151077', 'StopSequence': 1}, ...",TPE10001,2
3,TPE10002,3655,Taipei,王爺廟口,Wangye Temple Entrance,121.466640,25.088140,wsqqsp1m1,TPE10443,TPE10443,0,Taipei,225,225,225,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10002,11
4,TPE10002,3655,Taipei,王爺廟口,Wangye Temple Entrance,121.466640,25.088140,wsqqsp1m1,TPE10443,TPE159519,0,Taipei,225,225,225狗狗公車,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10002,11


In [40]:
def extract_mrt_stop(stop_name_zh_tw: str):
    if re.match(r'捷運(.)+', stop_name_zh_tw):
        res = True
    else:
        res = False
    return res

In [41]:
final_df['mrt_bus_stop'] = final_df['stop_name_zh_tw'].apply(extract_mrt_stop)

In [42]:
final_df.head()

,StopUID,StationID,City_x,stop_name_zh_tw,stop_name_en,PositionLon,PositionLat,GeoHash,RouteUID,SubRouteUID,Direction,City_y,route_name_zh_tw,route_name_en,subroute_name_zh_tw,subroute_name_en,stop_info,stop_in_route,order_in_route,mrt_bus_stop
0,TPE10000,3698,Taipei,蘆洲總站,Luzhou Bus Terminal,121.465846,25.089107,wsqqsp2dx,TPE10443,TPE10443,0,Taipei,225,225,225,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10000,10,False
1,TPE10000,3698,Taipei,蘆洲總站,Luzhou Bus Terminal,121.465846,25.089107,wsqqsp2dx,TPE10443,TPE159519,0,Taipei,225,225,225狗狗公車,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10000,10,False
2,TPE10001,8627,Taipei,南港分局(重陽),Nangang Police Dist.(Chongyang),121.596717,25.055561,wsqqx10p0,TPE10231,TPE10231,0,Taipei,民權幹線,Minquan Metro Bus,民權幹線,Minquan Metro Bus,"[{'StopUID': 'TPE151077', 'StopSequence': 1}, ...",TPE10001,2,False
3,TPE10002,3655,Taipei,王爺廟口,Wangye Temple Entrance,121.466640,25.088140,wsqqsp1m1,TPE10443,TPE10443,0,Taipei,225,225,225,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10002,11,False
4,TPE10002,3655,Taipei,王爺廟口,Wangye Temple Entrance,121.466640,25.088140,wsqqsp1m1,TPE10443,TPE159519,0,Taipei,225,225,225狗狗公車,225,"[{'StopUID': 'TPE148864', 'StopSequence': 1}, ...",TPE10002,11,False


In [43]:
def extract_school_stop(stop_name_zh_tw: str):
    # 國小
    if re.search('國小', stop_name_zh_tw):
        res = '國小'
    # 國中
    elif re.search('國中', stop_name_zh_tw):
        res = '國中'
    # 國高中
    elif re.search('中學', stop_name_zh_tw):
        res = '國高中'
    # 高中同等學歷
    elif re.search('高中', stop_name_zh_tw):
        res = '高中'
    elif re.search('女中', stop_name_zh_tw):
        res = '女子高中'
    elif re.search('高工', stop_name_zh_tw):
        res = '高工'
    elif re.search('商工', stop_name_zh_tw):
        res = '商工'
    elif re.search('商職', stop_name_zh_tw):
        res = '商職'
    elif re.search('高商', stop_name_zh_tw):
        res = '高商'
    # 大學、五專
    elif re.search('大學', stop_name_zh_tw):
        res = '大學'
    elif re.search('專科學校', stop_name_zh_tw):
        res = '專科學校'
    elif re.search('學院', stop_name_zh_tw):
        res = '學院'
    else:
        res = None
    return res 

In [44]:
final_df['school_bus_stop'] = final_df['stop_name_zh_tw'].apply(extract_school_stop)

In [45]:
# test = final_df[final_df['school_bus_stop'].notnull()]
# test.query("school_bus_stop=='專科學校'")['stop_name_zh_tw'].unique()

In [46]:
# final_df['stop_name_zh_tw'].unique()[0:1000]

## 經緯度兩點距離計算

haversine: 半正矢公式是一種根據兩點的經度和緯度來確定大圓上兩點之間距離的計算方法，在導航有著重要地位。

[Reference](https://zh.wikipedia.org/wiki/%E5%8D%8A%E6%AD%A3%E7%9F%A2%E5%85%AC%E5%BC%8F)

In [47]:

def haversine(lon1, lat1, lon2, lat2): # 經度1，緯度1，經度2，緯度2（十進制度數）
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # Using map method converts a degree value into radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
 
    # haversine公式
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371  # Radius of the earth in km
    return c * r * 1000

In [48]:
haversine(121.465845849213, 25.089107361013, 121.596717, 25.055561)

13698.65484492898

## 體育局列管臺北市各類運動場館

In [90]:
if __name__ == '__main__':
    response = request('get', 'https://data.taipei/api/getDatasetInfo/downloadResource?id=d2db2371-4bca-44cb-ac61-9a50e80c1873&rid=112521a2-7ee3-4c15-8495-9ddb3278ce75')
    if response.status_code == 200:
        decoded_content = response.content.decode('utf-8')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)
        gym_df = pd.DataFrame(my_list[1:], columns=my_list[0])
    else:
        print(f'request failed\n{response.status_code}')

In [91]:
	gym_df

,﻿編號,公民營,廠商名稱_市招,行政區,經營主體,聯絡電話,地址,經度,緯度
0,1,民營,臺北醫學大學,信義,臺北醫學大學,2739-9118 2236~1661分機2273,臺北市信義區吳興街250號,121.5627109,25.02661963
1,2,公營,信義運動中心,信義,社團法人中國青年救國團,8786-1911分機305,臺北市信義區松勤街100號,121.5667558,25.03180456
2,3,民營,World Gym健身俱樂部光復店,大安,香港商世界健身事業有限公司光復分公司,8771-3555,臺北市大安區忠孝東路4段330號B1,121.5574009,25.04113262
3,4,公營,博愛國小,信義,舞動陽光有限公司博愛營業所,8789-2525,臺北市信義區松仁路95巷20號,121.5711337,25.03644332
4,5,民營,GO GYM健身俱樂部永春館,信義,高盛健身事業股份有限公司永春分公司,2765-0700,臺北市信義區忠孝東路5段451號B1,121.576254,25.04105499
...,...,...,...,...,...,...,...,...,...
265,266,民營,國榮桌球古亭館,中正,國榮桌球館,2368-8818,台北市中正區和平西路1段34號2樓,121.5204201,25.02654815
266,267,民營,穜欣桌球館,士林,穜欣桌球館,2873-1818,臺北市士林區天母西路13巷9號,121.5277619,25.11919885
267,268,民營,圓山育樂中心,士林,中國體育振興企業股份有限公司,2881-2277,臺北市士林區中山北路5段6號,121.5256134,25.08236656
268,269,民營,長興極限空間,大同,集武躍有限公司,2552-9553,臺北市大同區重慶北路2段162號1樓,121.5134682,25.05894872


## 臺北市政府工務局公園路燈工程管理處(公園資料)

[Reference](https://parks.taipei/parks/index.php)

In [100]:
if __name__ == '__main__':
    response = request('get', 'https://parks.taipei/parks/api/')
    if response.status_code == 200:
        json = response.json()
    else:
        print(f'request failed\n{response.status_code}')

In [101]:
park_df = pd.DataFrame(json)

In [102]:
park_df = park_df[['pm_name', 'pm_lon', 'pm_lat', 'pm_location', 'pm_area', 'pm_sports', 'pm_recreation', 'pm_service', 'pm_transit', 'pm_type']]

In [103]:
park_df.head()

,pm_name,pm_lon,pm_lat,pm_location,pm_area,pm_sports,pm_recreation,pm_service,pm_transit,pm_type
0,七虎公園,121.50205994,25.136499405,育仁路108號（薇閣小學旁）,10098,游泳池,組合遊具,涼亭,"北投公園站:216區, 218, 218區, 218直達車, 223, 266, 602,2...",公園
1,七星公園,121.50302124,25.136619568,光明路、中山路交叉口（新北投捷運站）,12056,腹肌板、大轉輪、雙人肩關節康復器、三人轉腰器、雙人太空漫步器、雙人上肢牽引器,,公廁、生飲台,"北投公園站:216區, 218, 218區, 218直達車, 223, 266, 602,2...",公園
2,大豐公園,121.50373077,25.131940842,公館路13號旁,12391,運動設施、腹肌板、健騎機、大轉輪、三人扭腰器、太空漫步器、上肢牽引器,組合遊具、搖搖樂,溜冰場、網球場、籃球場、涼亭、生飲台1座、洗手台,"北投市場站 ：216區, 602, 小14, 小21, 市民小巴M2 \r\n",公園
3,中央公園,121.49992371,25.128099442,大興街與中央南路二段6巷口,4463,腹肌板、伸腰伸背器、太空漫步器、雙人上肢牽引器,組合遊具,,搭乘218至大興街口、223至慈后宮、搭乘紅線至捷運奇岩站,公園
4,中庸公園,121.50124359,25.138040543,位於北投區中庸里雙全街50號對面,5049,三人壓腿器、伸腰伸背器、雙人漫步器,組合遊具、搖搖樂,涼亭、生飲台,搭乘218、承德幹線至中和街，捷運紅線至北投站,公園


In [112]:
def extract_bus_from_pm_transit(text: str):
    text = re.sub(r'\s+', '', text)
    text = re.sub(r'(.)+(:|：)', '', text)
    text = text.split(',')
    res = list(map(lambda x: x.strip(), text))
    return res

In [113]:
park_df['nearby_bus'] = park_df['pm_transit'].apply(extract_bus_from_pm_transit)

In [114]:
park_df.head()

,pm_name,pm_lon,pm_lat,pm_location,pm_area,pm_sports,pm_recreation,pm_service,pm_transit,pm_type,nearby_bus
0,七虎公園,121.50205994,25.136499405,育仁路108號（薇閣小學旁）,10098,游泳池,組合遊具,涼亭,"北投公園站:216區, 218, 218區, 218直達車, 223, 266, 602,2...",公園,"[216區, 218, 218區, 218直達車, 223, 266, 602, 230, ..."
1,七星公園,121.50302124,25.136619568,光明路、中山路交叉口（新北投捷運站）,12056,腹肌板、大轉輪、雙人肩關節康復器、三人轉腰器、雙人太空漫步器、雙人上肢牽引器,,公廁、生飲台,"北投公園站:216區, 218, 218區, 218直達車, 223, 266, 602,2...",公園,"[216區, 218, 218區, 218直達車, 223, 266, 602, 230, ..."
2,大豐公園,121.50373077,25.131940842,公館路13號旁,12391,運動設施、腹肌板、健騎機、大轉輪、三人扭腰器、太空漫步器、上肢牽引器,組合遊具、搖搖樂,溜冰場、網球場、籃球場、涼亭、生飲台1座、洗手台,"北投市場站 ：216區, 602, 小14, 小21, 市民小巴M2 \r\n",公園,"[216區, 602, 小14, 小21, 市民小巴M2]"
3,中央公園,121.49992371,25.128099442,大興街與中央南路二段6巷口,4463,腹肌板、伸腰伸背器、太空漫步器、雙人上肢牽引器,組合遊具,,搭乘218至大興街口、223至慈后宮、搭乘紅線至捷運奇岩站,公園,[搭乘218至大興街口、223至慈后宮、搭乘紅線至捷運奇岩站]
4,中庸公園,121.50124359,25.138040543,位於北投區中庸里雙全街50號對面,5049,三人壓腿器、伸腰伸背器、雙人漫步器,組合遊具、搖搖樂,涼亭、生飲台,搭乘218、承德幹線至中和街，捷運紅線至北投站,公園,[搭乘218、承德幹線至中和街，捷運紅線至北投站]


## 臺北捷運車站出入口座標

In [54]:

if __name__ == '__main__':
    a = Auth(app_id, app_key)
    response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Rail/Metro/StationExit/TRTC?$format=json', headers= a.get_auth_header())
    if response.status_code == 200:
        json = response.json()
    else:
        print(f'request failed\n{response.status_code}')

In [55]:
mrt_exit_df = pd.DataFrame(json)

In [56]:
mrt_exit_df

,StationID,StationName,ExitID,ExitName,ExitPosition,LocationDescription,Stair,Escalator,Elevator,SrcUpdateTime,UpdateTime,VersionID
0,BL01,"{'Zh_tw': '頂埔', 'En': 'Dingpu'}",1,"{'Zh_tw': '頂埔站出口1', 'En': 'Dingpu Exit 1'}","{'PositionLon': 121.418218, 'PositionLat': 24....",中央路4段約100號旁,True,2,True,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
1,BL01,"{'Zh_tw': '頂埔', 'En': 'Dingpu'}",2,"{'Zh_tw': '頂埔站出口2', 'En': 'Dingpu Exit 2'}","{'PositionLon': 121.419, 'PositionLat': 24.959...",中央路4段，近鴻海精密工業,True,2,False,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
2,BL01,"{'Zh_tw': '頂埔', 'En': 'Dingpu'}",3,"{'Zh_tw': '頂埔站出口3', 'En': 'Dingpu Exit 3'}","{'PositionLon': 121.4196, 'PositionLat': 24.95...",中央路4段，近嵿埔之星科技廣場,True,1,True,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
3,BL01,"{'Zh_tw': '頂埔', 'En': 'Dingpu'}",4,"{'Zh_tw': '頂埔站出口4', 'En': 'Dingpu Exit 4'}","{'PositionLon': 121.4201, 'PositionLat': 24.96...",中央路4段62號旁,True,0,True,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
4,BL02,"{'Zh_tw': '永寧', 'En': 'Yongning'}",1,"{'Zh_tw': '永寧站出口1', 'En': 'Yongning Exit 1'}","{'PositionLon': 121.435254, 'PositionLat': 24....",中央路3段98巷口,True,1,True,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
...,...,...,...,...,...,...,...,...,...,...,...,...
382,Y18,"{'Zh_tw': '頭前庄', 'En': 'Touqianzhuang'}",3,"{'Zh_tw': '頭前庄站出口3', 'En': 'Touqianzhuang Exit...","{'PositionLon': 121.462247, 'PositionLat': 25....",中正路、近中正路61巷,True,2,False,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
383,Y18,"{'Zh_tw': '頭前庄', 'En': 'Touqianzhuang'}",4,"{'Zh_tw': '頭前庄站出口4', 'En': 'Touqianzhuang Exit...","{'PositionLon': 121.462242, 'PositionLat': 25....",中正路、近中正路56巷,True,2,False,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
384,Y19,"{'Zh_tw': '幸福', 'En': 'Xingfu'}",1,"{'Zh_tw': '幸福站出口1', 'En': 'Xingfu Exit 1'}","{'PositionLon': 121.460146, 'PositionLat': 25....",思源路與思源路296巷交叉口,True,1,True,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2
385,Y19,"{'Zh_tw': '幸福', 'En': 'Xingfu'}",2,"{'Zh_tw': '幸福站出口2', 'En': 'Xingfu Exit 2'}","{'PositionLon': 121.460593, 'PositionLat': 25....",思源路與幸福東路交叉口,True,0,False,2020-01-31T14:00:00+08:00,2020-05-20T12:00:00+08:00,2


## 臺北捷運車站出入口無障礙電梯、無障礙坡道GPS座標
[Reference](https://data.gov.tw/dataset/128414)

In [57]:
if __name__ == '__main__':
    response = request('get', 'https://data.taipei/api/getDatasetInfo/downloadResource?id=0a3bb422-9eb5-459b-a9d4-138456516183&rid=7a3a6a6d-3319-41f3-a422-df537db2c25b')
    if response.status_code == 200:
        decoded_content = response.content.decode('Big5')
        cr = csv.reader(decoded_content.splitlines(), delimiter=',')
        my_list = list(cr)
        mrt_barrier_free_df = pd.DataFrame(my_list[1:], columns=my_list[0])
    else:
        print(f'request failed\n{response.status_code}')

In [58]:
mrt_barrier_free_df

,項次,出入口電梯/無障礙坡道名稱,出入口編號,經度,緯度
0,1,動物園站出口電梯1,出口1,121.5797157,24.9982136
1,2,動物園站出口電梯2,出口2,121.5795233,24.99809
2,3,木柵站出口無障礙坡道,單一出口,121.5732164,24.9980328
3,4,萬芳社區站出口無障礙坡道,單一出口,121.5685909,24.9984689
4,5,萬芳醫院站出口電梯,單一出口,121.5577226,24.999444
...,...,...,...,...,...
178,179,中原站出口電梯,單一出口,121.4846089,25.0082755
179,180,板新站出口電梯,單一出口,121.4724853,25.0144302
180,181,新埔民生站出口電梯,單一出口,121.4668182,25.0261323
181,182,幸褔站出口電梯,出口1,121.460188,25.049854


## 臺北市公共自行車站點資料服務

In [59]:
if __name__ == '__main__':
    if __name__ == '__main__':
        a = Auth(app_id, app_key)
        response = request('get', 'https://ptx.transportdata.tw/MOTC/v2/Bike/Station/Taipei?&$format=JSON', headers= a.get_auth_header())
        if response.status_code == 200:
            json = response.json()
        else:
            print(f'request failed\n{response.status_code}')

In [60]:
bike_station_df = pd.DataFrame(json)

In [61]:
bike_station_df

,StationUID,StationID,AuthorityID,StationName,StationPosition,StationAddress,BikesCapacity,SrcUpdateTime,UpdateTime
0,TPE0001,0001,TPE,"{'Zh_tw': '捷運市政府站(3號出口)', 'En': 'MRT Taipei Ci...","{'PositionLon': 121.567904444, 'PositionLat': ...","{'Zh_tw': '忠孝東路/松仁路(東南側)', 'En': 'The S.W. sid...",180,2021-05-28T16:13:42+08:00,2021-05-28T16:15:18+08:00
1,TPE0002,0002,TPE,"{'Zh_tw': '捷運國父紀念館站(2號出口)', 'En': 'MRT S.Y.S M...","{'PositionLon': 121.55742, 'PositionLat': 25.0...","{'Zh_tw': '忠孝東路四段/光復南路口(西南側)', 'En': 'Sec,4. Z...",48,2021-05-28T16:13:21+08:00,2021-05-28T16:15:18+08:00
2,TPE0003,0003,TPE,"{'Zh_tw': '台北市政府', 'En': 'Taipei City Hall'}","{'PositionLon': 121.565169444, 'PositionLat': ...","{'Zh_tw': '台北市政府東門(松智路) (鄰近信義商圈/台北探索館)', 'En':...",40,2021-05-28T16:13:23+08:00,2021-05-28T16:15:18+08:00
3,TPE0004,0004,TPE,"{'Zh_tw': '市民廣場', 'En': 'Citizen Square'}","{'PositionLon': 121.562325, 'PositionLat': 25....",{'Zh_tw': '市府路/松壽路(西北側)(鄰近台北101/台北世界貿易中心/台北探索館...,60,2021-05-28T16:13:26+08:00,2021-05-28T16:15:18+08:00
4,TPE0005,0005,TPE,"{'Zh_tw': '興雅國中', 'En': 'Xingya Jr. High School'}","{'PositionLon': 121.5686639, 'PositionLat': 25...","{'Zh_tw': '松仁路/松仁路95巷(東南側)(鄰近信義商圈/台北信義威秀影城)', ...",60,2021-05-28T16:13:34+08:00,2021-05-28T16:15:18+08:00
...,...,...,...,...,...,...,...,...,...
394,TPE0401,0401,TPE,"{'Zh_tw': '南京新生路口', 'En': 'Nanjing & Xinsheng ...","{'PositionLon': 121.527661, 'PositionLat': 25....",{'Zh_tw': '新生北路一段 / 南京東路一段口(橋墩下方)(鄰近林森公園/康樂公園)...,34,2021-05-28T16:13:38+08:00,2021-05-28T16:15:18+08:00
395,TPE0402,0402,TPE,"{'Zh_tw': '下灣公園', 'En': 'Siawan Park'}","{'PositionLon': 121.595611, 'PositionLat': 25....","{'Zh_tw': '民權東路六段206巷 / 民權東路六段190巷75弄口', 'En':...",28,2021-05-28T16:13:38+08:00,2021-05-28T16:15:18+08:00
396,TPE0403,0403,TPE,"{'Zh_tw': '捷運內湖站(1號出口)', 'En': 'MRT Neihu Sta....","{'PositionLon': 121.593929, 'PositionLat': 25....","{'Zh_tw': '成功路四段182巷 / 成功路四段182巷6弄口(東南側)', 'En...",28,2021-05-28T16:13:30+08:00,2021-05-28T16:15:18+08:00
397,TPE0404,0404,TPE,"{'Zh_tw': '民族延平路口', 'En': 'Minzu & Yanping Int...","{'PositionLon': 121.510569, 'PositionLat': 25....","{'Zh_tw': '民族西路 310 號前方', 'En': 'No.310, Minzu...",30,2021-05-28T16:13:41+08:00,2021-05-28T16:15:18+08:00
